# Xsarsea example

The Normalized Radar Cross Section (sigma0) as computed from Level-1 SAR data can be detrended in the case of ocean scenes. 

The goal is to remove the averaged trend (decreasing) of the NRCS with (increasing) incidence angle observed for acquisitions over ocean. 

The detrend maximizes the contrasts in the image due to geophysical phenomena and improves the visualization experience of ocean scenes. 

**Sigma0_detrend** is also termed **image roughness** or **nice display**.

In [ ]:
import xsar
import xsarsea
import os
import datetime

In [ ]:
# use holoviews for plots
import bokeh.io
bokeh.io.output_notebook()
import holoviews as hv
hv.extension('bokeh')
from holoviews.operation.datashader import datashade,rasterize

In [ ]:
# optional debug message
import logging
logging.basicConfig()
logging.getLogger('xsarsea').setLevel(logging.INFO) # .setLevel(logging.DEBUG) for more messages

## read the dataset with xsar

In [ ]:
# get test file. You can replace with an path to other SAFE
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE')
#filename = xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE')
filename

In [ ]:
# open the dataset with xsar
sar_ds = xsar.open_dataset(filename, resolution='1000m')
sar_ds[['sigma0','incidence']]


## Sigma0 detrending

Sigma0 detrending is done by  [xsarsea.sigma0_detrend](../basic_api.rst#xsarsea.sigma0_detrend) function

As the resulting xarray dataset have the same coordinates as the original sigma0, we can add a `sigma0_detrend` variable to the dataset.

In [ ]:
sar_ds['sigma0'] = sar_ds.sigma0
sar_ds['sigma0_detrend'] = xsarsea.sigma0_detrend(sar_ds.sigma0, sar_ds.incidence)

In [ ]:
(
    hv.Image(sar_ds.sigma0.sel(pol='VV')).opts(title="original sigma0") \
    + hv.Image(sar_ds.sigma0_detrend.isel(pol=0)).opts(title="detrended sigma0")
).opts(hv.opts.Image(cmap='gray', clim=(0,0.4)))